In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

CCAL version 0.9.1 @ /home/kwatme/github/ccal/ccal/__init__.py


In [3]:
signal = pd.read_table(PATH["feature_x_sample.signal.tsv"], index_col=0)

signal.columns.name = SETTING["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
k_nmfcc = ccal.nmf_consensus_cluster_with_ks(
    signal,
    SETTING["nmf_ks"],
    n_job=SETTING["max_n_job"],
    n_clustering=SETTING["nmf_n_clustering"],
    plot_df=signal.size < SETTING["plot_heat_map_max_size"],
    plot_w=signal.shape[0] < SETTING["plot_cluster_max_size"],
    plot_h=signal.shape[1] < SETTING["plot_cluster_max_size"],
    directory_path=PATH["nmf/"],
)

NMFCC with K=2 ...
	(K=2) 1/10 ...
NMFCC with K=3 ...
	(K=3) 1/10 ...
NMFCC with K=4 ...
	(K=4) 1/10 ...
NMFCC with K=5 ...
NMFCC with K=6 ...
	(K=6) 1/10 ...
	(K=5) 1/10 ...
Plotting w ...
file:///media/kwatme/CarrotCake/project/ribosome/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/2/w.html


	(K=2) 2/10 ...
	(K=2) 3/10 ...


In [ ]:
for k in SETTING["nmf_ks"]:

    for w_or_h, axis in (("w", 0), ("h", 1)):

        df = pd.read_table("{}/{}/{}.tsv".format(PATH["nmf/"], k, w_or_h), index_col=0)

        ccal.make_comparison_panel(
            df,
            df,
            axis=axis,
            title="NMF K={} {}".format(k, w_or_h.title()),
            name_0="Factor",
            name_1="Factor",
        )